In [37]:
import pandas as pd
import numpy as np
import openpyxl

# Define the diff function to show the changes in each field

In [38]:
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)

In [39]:
cd C:\Users\User1\OneDrive\Desktop\Comparisions

C:\Users\User1\OneDrive\Desktop\Comparisions


# Read in the excel file and assign a new column with values to keep track of origin of data

In [40]:
df1=pd.read_excel('Utilization Report for January 2020.xlsx', na_values=['NaN'])
df2=pd.read_excel('PowerBI version 1 data for 2020 comparision.xlsx', na_values=['NaN'])
df1['version'] = "old"
df2['version'] = "new"

In [9]:
#df1.head()

In [10]:
#df2.head()

# Rename the Columns to the same name for both data frames

In [11]:
#df1=df1.rename({'Name':'Full Name','LY Billable':'LY Billable Hours','Total Hours':'LY Total Hours','Utilization': 'LY Util %'}, axis=1)
#df1.head(2)

# Columns have different value types, converting the strings (objects) to numeric

In [13]:
#cols = ['LY Billable Hours', 'LY Total Hours']
#df1[cols] = df1[cols].apply(pd.to_numeric, errors='coerce', axis=1)
#df1.head()

# Confirm data types

In [46]:
df1.dtypes

Department            object
Full Name             object
LY Billable Hours    float64
LY Total Hours       float64
LY Util %            float64
version               object
dtype: object

In [47]:
df2.dtypes

Department            object
Full Name             object
LY Billable Hours    float64
LY Total Hours       float64
LY Util %            float64
version               object
dtype: object

In [48]:
df1=df1.fillna(0)
df2=df2.fillna(0)

# We use the account numbers as the keys to check what is added, dropped and potentially changed

# Using sets makes the deduping easy and we can use set operations to figure out groupings

In [49]:
old_data_all = set(df1['Full Name'])
new_data_all = set(df2['Full Name'])

dropped_data = old_data_all - new_data_all
added_data = new_data_all - old_data_all

# Join all the data together and ignore indexes so it all gets concatenated

In [50]:
all_data = pd.concat([df1,df2],ignore_index=True)

In [14]:
#all_data.head()

# Let's see what changes in the main columns we care about
# Change drop_duplicates syntax: keep=last

In [52]:
changes = all_data.drop_duplicates(subset=["Department","Full Name", 
                                           "LY Billable Hours", "LY Total Hours", 
                                           "LY Util %"], keep='last')

In [15]:
#changes.head()

# Get all the duplicate rows

In [54]:
dupe_accts = changes[changes['Full Name'].duplicated() == True]['Full Name'].tolist()
dupes = changes[changes["Full Name"].isin(dupe_accts)]

In [16]:
#dupes

# Pull out the old and new data into separate dataframes

In [56]:
change_new = dupes[(dupes["version"] == "new")]
change_old = dupes[(dupes["version"] == "old")]

# Drop the temp columns - we don't need them now

In [20]:
#change_new = change_new.drop(['version'], axis=1)
#change_old = change_old.drop(['version'], axis=1)

# Index on the Full Name

In [ ]:
#change_new.set_index('Full Name', inplace=True)
#change_old.set_index('Full Name', inplace=True)

In [72]:
#df_all_changes = pd.concat([change_old, change_new],
                           #axis='columns',
                           #keys=['old', 'new'],
                           #join='outer')

In [27]:
#df_all_changes.head()

In [3]:
#df_all_changes = df_all_changes.swaplevel(axis='columns')[change_new.columns[0:]]

In [4]:
#df_all_changes

In [5]:
#df_changed = df_all_changes.groupby(level=0,axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))
#df_changed = df_changed.reset_index()

In [6]:
#df_changed

# Diff'ing is done, we need to get a list of removed and added items

In [7]:
#df_removed = changes[changes["Full Name"].isin(dropped_data)]
#df_removed

In [8]:
#df_added = changes[changes["Full Name"].isin(added_data)]
#df_added

# Save the changes to excel but only include the columns we care about

In [137]:
output_columns = ["Department", "Full Name", "LY Billable Hours", "LY Total Hours", "LY Util %"]
writer = pd.ExcelWriter("my-diff.xlsx")
df_changed.to_excel(writer,"changed", index=False, columns=output_columns)
df_removed.to_excel(writer,"removed",index=False, columns=output_columns)
df_added.to_excel(writer,"added",index=False, columns=output_columns)
writer.save()